#### Rapid tests outcomes
Day 0:

    * Test 1: Positive
    * Test 2: Negative
    * Test 3: Negative
Day 1:

    * Test 4: Negative
    
#### Rapid test (VivaDiag Pro) Metrics
* Accuracy = 98.94%
* Specificity = 99.99%
* Sensitivity = 96.12%


### Greece Covid prevelence
Daily active cases = 200.000 (Source: https://www.worldometers.info/coronavirus/country/greece/)
Greece Population = 10.72M
Covid prevelence = Daily active cases / Greece Population = 200.000/ 10.720.000 = 0.0186 or 1.86%

    
#### Question
What is the probability for the subject to have SARS-CoV-2?

## Estimate Probabilities for each test

#### Notations

**Hypothesis**

Ho: Disease Free

H1: Have Disease (SARS-CoV-2)

**Outcomes**

**+** : Positive test outcome

**-** : Negative test outcome

### Propability estimation maths

#### 1st Test: Estimate the probability of has SARS-CoV-2 (H1) given a positive test (+)
P(H1 | +) = P(+ | H1) * P(H1) / P(+) (Bayes Rule)

P(H1) = 0.0186
P(Ho) = 1 - P(H1)
P(+ | H1) = Sensitivity
P(+ | Ho) = 1-Specificity
P(+) = TruePositive + FalsePositive = P(+ | H1)*P(H1) + P(+ | Ho)*P(Ho)

Finally
P(H1 | +) = 0.9945

-----------------------

#### 2nd Test: Estimate the probability of has SARS-CoV-2 (H1) given a negative test (-)
P(H1 | -) = P(- | H1) * P(H1) / P(-) (Bayes Rule)

P(H1) = 0.645 (Previous test probability)
P(Ho) = 1 - P(H1)
P(- | H1) = 1 - Sensitivity
P(- | Ho) = 1 - P(+ | Ho) = 1 - (1-Specificity) = Specificity
P(-) = TrueNegative + FalseNegative = P(- | Ho)*P(Ho) + P(- | H1)*P(H1)

Finally
P(- | Ho) = 0.8760

etc ...

In [16]:
def estimate_has_covid_given_positive_test(specificity, sensitivity, P_H1):
    """
    * Hypothesis: Has SARS-CoV-2 (H1)
    * Test Outcome: Positive (+)
    * Formula: P(H1 | +) = P(+ | H1) * P(H1) / P(+) (Bayes Rule)
    """
    P_evidence = sensitivity*P_H1 + (1-specificity)*(1-P_H1)
    return sensitivity * P_H1 / P_evidence


def estimate_has_covid_given_negative_test(specificity, sensitivity, P_H1):
    """
    * Hypothesis: Has SARS-CoV-2 (H1)
    * Test Outcome: Negative (-)
    * Formula: P(H1 | -) = P(- | H1) * P(H1) / P(-) (Bayes Rule)
    """
    P_evidence = specificity*(1-P_H1) + (1-sensitivity)*P_H1
    return (1-sensitivity)*P_H1 / P_evidence

def run_multiple_negative_tests(specificity, sensitivity, number_of_tests, P_H1):
    results = []
    for _ in range(number_of_tests):
        P_H1 = estimate_has_covid_given_negative_test(specificity, sensitivity, P_H1)
        results.append(P_H1)
    return results


In [22]:
specificity = 0.9999
sensitivity = 0.9612
P_H1 = 0.0186 # Initial prior probability --> Population prevelence


positive_test = estimate_has_covid_given_positive_test(specificity, sensitivity, P_H1)
negative_tests = run_multiple_negative_tests(specificity, sensitivity, number_of_tests=3, P_H1=positive_test)

In [43]:
import plotly.express as px

test_cases = ['Test 1: +', 'Test 2: -', 'Test 3: -', 'Test 4: -']
test_results = [positive_test] + negative_tests
test_results_prc = [p*100 for p in test_results]
fig = px.bar(x=test_cases,
             y=test_results_prc,
             title='Rapid Tests Probability Evolution',
             text_auto=True,
             template='plotly_dark')

fig.update_xaxes(title_text='Rapid Tests Outcomes')
fig.update_yaxes(title_text='Probability to has SARS-CoV-2 (%)')
fig.show()

## Point-of-care Tests
Rapid tests which performed out of Laboratory. For example pharmacies, homes etc.

```
Realistic Sensitivity = (Ideal Sensitivity) * 0.789
```

Source: https://www.healthline.com/health/how-accurate-are-rapid-covid-tests

In [47]:
P_H1 = 0.0186 # Initial prior probability --> Population prevelence
sensitivity = 0.9612
sensitivity_values = [sensitivity - 0.02*i for i in range(25)]

positive_test_values = []
for sensitivity in sensitivity_values:
    positive_test_values.append(estimate_has_covid_given_positive_test(specificity, sensitivity, P_H1))

sensitivity_values_prc = [p*100 for p in sensitivity_values]
positive_test_values_prc = [p*100 for p in positive_test_values]
fig = px.line(x=sensitivity_values_prc,
              y=positive_test_values_prc,
             title='Rapid Test Sensitivity VS Has SARS-CoV-2 Probability',
             template='plotly_dark')

fig.update_xaxes(title_text='Rapid Test Sensitivity (%)')
fig.update_yaxes(title_text='Probability to has SARS-CoV-2 (%)')
fig.add_vline(x=78.9, line_width=3, line_dash="dash", line_color="red")
fig.show()

Re-estimate the probability after the four tests

In [48]:
specificity = 0.9999
sensitivity = 0.9612 * 0.789
P_H1 = 0.0186 # Initial prior probability --> Population prevelence


positive_test = estimate_has_covid_given_positive_test(specificity, sensitivity, P_H1)
negative_tests = run_multiple_negative_tests(specificity, sensitivity, number_of_tests=3, P_H1=positive_test)

test_cases = ['Test 1: +', 'Test 2: -', 'Test 3: -', 'Test 4: -']
test_results = [positive_test] + negative_tests
test_results_prc = [p*100 for p in test_results]
fig = px.bar(x=test_cases,
             y=test_results_prc,
             title='Rapid Tests Probability Evolution',
             text_auto=True,
             template='plotly_dark')

fig.update_xaxes(title_text='Rapid Tests Outcomes')
fig.update_yaxes(title_text='Probability to has SARS-CoV-2 (%)')
fig.show()